In [ ]:
import os
import numpy as np
#import xlrd
import pickle as pickle
import gudhi as gd  
import gudhi.representations
#from pylab import *
from numpy import *
from sklearn.metrics import pairwise_distances
import csv
from tabulate import tabulate
#include these two line if you are using Ipython
import ripser 
import persim 
from persim.persistent_entropy import *
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import figure

In [ ]:
def get_data(Gweek): #my_path is a global variable
    fname = ''.join([my_path, Gweek])
    with open(fname, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        headers = reader.fieldnames
        fullData=[]
        for row in reader:
            fullData.append(row)
           
    return fullData, headers

In [ ]:
def get_clean_data(col):    
    #skip empty rows with empty cells... one collumn at the time
    new_rows=[]
    emptyCell = False  #assume there are no empty cells 

    for i in range(rowLen_full):          
        y=rows_fullData[i][col]
        if y == "":
             #print("row=", i, "-- skipping empty cell for WOMAN ", col)
            emptyCell=True #found one
            continue  #check for more
        new_rows.append(rows_fullData[i])
        
    return new_rows    

In [ ]:
def get_X_data_points(rowsData, w):
        rowsLen=len (rowsData)
        X = np.empty([rowsLen,2], dtype=np.float64)
        for i in range(rowsLen):
            x=rowsData[i]['time']
            y=rowsData[i][w]
            X[i,:] = [x, y]
        
        return X

In [ ]:
def saveRipsPersistenceDiagram(pd0,pd1,txt):
        fpath = ''.join([my_path, 'RipsPersistenceDiagrams'])
        woman_str = ''.join(['PD_', str(txt)])
        filename = woman_str + ".png"

        plt.figure(figsize=(15,8))
        plt.title("Persistence Diagram For (%s)"%(txt) ) #add woman num
        persim.plot_diagrams([pd0,pd1],labels=['$H_0$', '$H_1$'])
   
        plt.savefig(os.path.join(fpath, filename)) 
        plt.close()
    
        return;

In [ ]:
def savePlotScatter(x,w):
        fpath = ''.join([my_path, 'ScatterPlots'])
        woman_str = ''.join(['ScatterPlot_', str(w)])
        filename = woman_str + ".png"

        plt.figure(figsize=(15,8))
        plt.scatter(x[:, 0], x[:,1], s=50, alpha=0.3)
        plt.title("Scatter Plot Woman (%s)"%(w) ) #add woman num
        plt.xlabel('time frame')
        plt.ylabel('activity')
   
        plt.savefig(os.path.join(fpath, filename)) 
        plt.close()
    
        return;

In [ ]:
def compute_BN_distance_all_women(PD_list):
    #PD_list is a list of persistence diagrams
    s=num_of_women
    bnd= np.ones( (s, s) , dtype=np.float64)

    #compute bottle neck distance for all persistence diagrams 
    for i in range(0, s):
        for j in range(0,s):
            #bnd[i,j] = persim.bottleneck(PD_list[i], PD_list[j]) #very slow 
            bnd[i,j] = gd.bottleneck_distance(PD_list[i], PD_list[j]) #faster
            print("BN[",i,",",j,"]=", "%0.2f"% bnd[i,j])
        
    return bnd

In [ ]:
def get_rips_pd_greedyPermutation(Points):
    
        tik = time.time()
        result = ripser.ripser(Points, n_perm=subsample_pts) #default is maxdim=1, n_perm to get greedy permutation
        tok = time.time() #computes the amount of time it took to run one persistence diagram
        print("greedy permutation Elapsed Time", "%0.2f"%(tok-tik))

        dgms_sub = result['dgms']
        idx_perm = result['idx_perm'] #idx_perm returns the new points ex) idx_perm[0] idx_perm[1]
        r_cover = result['r_cover']
    
        return dgms_sub, idx_perm, r_cover

In [ ]:
def get_all_persistence_diagrams_using_greedy_permutation():
    
    pd0_list=[] #for dim 0
    pd1_list=[] #for dim 1
    perm0_list = [] #for dim 0
    perm1_list = [] #for dim 1
    
    for w in (women_set): 
        print ("Computing Persistence Diagram for ", w )
        row_data=get_clean_data(w) #skip row with empty cells
        
        if (len(row_data) < subsample_pts):
            print("subsample greater than full data for  woman ", w)
            Points= get_X_data_points(row_data, w)
            result = ripser.ripser(Points) #no need to use greedy permutation
            dgms = result['dgms']
            diag0=dgms[0]
            diag1=dgms[1]
        else:       
            Points= get_X_data_points(row_data, w)
            [dgms_sub, idx_perm, r_cover] = get_rips_pd_greedyPermutation(Points)
            diag0=dgms_sub[0]
            diag1=dgms_sub[1]
            
            idx_perm0 = idx_perm[0] #permutation points for dimension 0
            idx_perm1 = idx_perm[1] #permutation points for dimension 1
        
            pd0_list.append(diag0)
            pd1_list.append(diag1)
            perm0_list.append(idx_perm0)
            perm1_list.append(idx_perm1)
        
    return pd0_list, pd1_list, perm0_list, perm1_list


In [ ]:
def save_all_scatter_plots():  
    print ("Saving Scatter Plots in PNG ")
    #save scatter plots to file for all wowmen  
    for w in (women_set):
        row_data=get_clean_data(w) #skip empty cells
        pts= get_X_data_points(row_data, w)
        savePlotScatter( pts,w) 
    return       


In [ ]:
def save_all_persistence_diagrams(pd0_lst, pd1_lst):
    print ("Saving Persistence Diagrams in PNG ")
    
    for k, w in enumerate(women_set):
        saveRipsPersistenceDiagram(pd0_lst[k],pd1_lst[k], w)
            
    return;  


In [ ]:
def save_Bottleneck_dist(file_name,BN_dist,col_names, rowids):
    print ("Saving BottleNeck Distances ")
    fpath = ''.join([my_path, file_name])
    with open(fpath, 'w',encoding="utf-8" ) as fp:
        fp.write(tabulate(BN_dist, col_names, tablefmt="tsv",showindex=rowids,floatfmt="0.2f"))


In [ ]:
def calculate_BN_dist_women_against_avg(PD_avg, pd_list): 
    #is the persistence diagram for the AVERAGE Collumn, and pd_list is the list of persistence diagrams
    s=num_of_women
    bnd= np.ones((s), dtype=np.float64)
    for i in range(0, s):   
        #bnd[i,j] = persim.bottleneck(PD_list[i], PD_list[j]) #very slow
        bnd[i] = gd.bottleneck_distance(PD_avg, pd_list[i])  #faster
        print("BN_vs_avg[",i,"]=", "%0.2f"% bnd[i])
    
    return bnd

In [ ]:
def calculate_BN_dist_women_against_median(PD_median, pd_list): 
    #is the persistence diagram for the MEDIAN Collumn, and pd_list is the list of persistence diagrams
    s=num_of_women
    bnd= np.ones((s), dtype=np.float64)
    for i in range(0, s):   
        #bnd[i,j] = persim.bottleneck(PD_list[i], PD_list[j]) #very slow
        bnd[i] = gd.bottleneck_distance(PD_median, pd_list[i])  #faster
        print("BN_vs_median[",i,"]=", "%0.2f"% bnd[i])
    
    return bnd

In [ ]:
def plot_BN_women_against_avg(bnd_avg, file_name, dim):
 
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1
    plt.figure(figsize=(15, 8))
    plt.title("PD_women vs PD_Avg   Dimension %i"%dim)
    plt.xlabel('Persistence diagram index ')
    plt.ylabel('Bottleneck Distance')
    plt.scatter(x, bnd_avg, s=30, alpha=0.9, color='blue')
    #plt.plot(x, bnd_avg)
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], bnd_avg[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[6, 8, 10, 12, 14, 15, 16, 17, 26, 32, 33, 36, 38, 41, 53, 54, 60, 61, 71, 75, 82, 87] #manually enter these values
    plt.plot(3, bnd_avg[3],'ro') 
    plt.plot(4, bnd_avg[4],'ro') 
    plt.plot(6, bnd_avg[6],'ro') #'pe' stands for persistence entropy 
    plt.plot(7, bnd_avg[7],'ro')
    plt.plot(9, bnd_avg[9],'ro')
    plt.plot(10, bnd_avg[10],'ro')
    plt.plot(11, bnd_avg[11],'ro')
    plt.plot(16, bnd_avg[16],'ro')
    plt.plot(17, bnd_avg[17],'ro')
    plt.plot(18, bnd_avg[18],'ro')
    plt.plot(19, bnd_avg[19],'ro')
    plt.plot(20, bnd_avg[20],'ro')
    plt.plot(22, bnd_avg[22],'ro')
    plt.plot(26, bnd_avg[26],'ro')
    plt.plot(27, bnd_avg[27],'ro')
    plt.plot(30, bnd_avg[30],'ro')
    plt.plot(31, bnd_avg[31],'ro')
    plt.plot(33, bnd_avg[33],'ro')
    plt.plot(34, bnd_avg[34],'ro')
    plt.plot(38, bnd_avg[38],'ro')
    plt.plot(42, bnd_avg[42],'ro')
    

    if(dim==0): 
        plt.plot(36, 550,'bo') #blue
        plt.plot(36, 530,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 550,"normal pregnacy")
        plt.text(37, 530,"bad pregnacy: all features")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 23,'bo') #blue
        plt.plot(36, 22,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 23,"normal pregnacy")
        plt.text(37, 22,"bad pregnacy: all features")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    
    print(bnd_avg)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()
    
    return


In [ ]:
def plot_BN_women_against_median(bnd_median, file_name, dim):
 
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1
    plt.figure(figsize=(15, 8))
    plt.title("PD_women vs PD_Median   Dimension %i"%dim)
    plt.xlabel('Persistence diagram index ')
    plt.ylabel('Bottleneck Distance')
    plt.scatter(x, bnd_median, s=30, alpha=0.9, color='blue')
    #plt.plot(x, bnd_avg)
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], bnd_median[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[6, 8, 10, 12, 14, 15, 16, 17, 26, 32, 33, 36, 38, 41, 53, 54, 60, 61, 71, 75, 82, 87] #manually enter these values
    plt.plot(3, bnd_median[3],'ro') 
    plt.plot(4, bnd_median[4],'ro') 
    plt.plot(6, bnd_median[6],'ro') #'pe' stands for persistence entropy 
    plt.plot(7, bnd_median[7],'ro')
    plt.plot(9, bnd_median[9],'ro')
    plt.plot(10, bnd_median[10],'ro')
    plt.plot(11, bnd_median[11],'ro')
    plt.plot(16, bnd_median[16],'ro')
    plt.plot(17, bnd_median[17],'ro')
    plt.plot(18, bnd_median[18],'ro')
    plt.plot(19, bnd_median[19],'ro')
    plt.plot(20, bnd_median[20],'ro')
    plt.plot(22, bnd_median[22],'ro')
    plt.plot(26, bnd_median[26],'ro')
    plt.plot(27, bnd_median[27],'ro')
    plt.plot(30, bnd_median[30],'ro')
    plt.plot(31, bnd_median[31],'ro')
    plt.plot(33, bnd_median[33],'ro')
    plt.plot(34, bnd_median[34],'ro')
    plt.plot(38, bnd_median[38],'ro')
    plt.plot(42, bnd_median[42],'ro')
    

    if(dim==0): 
        plt.plot(36, 600,'bo') #blue
        plt.plot(36, 550,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 600,"normal pregnacy")
        plt.text(37, 550,"bad pregnacy: all features")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 23,'bo') #blue
        plt.plot(36, 22,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 23,"normal pregnacy")
        plt.text(37, 22,"bad pregnacy: all features")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(bnd_median)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()
    
    return


In [ ]:
#based off birthweight
def save_persistence_entropy_birthweight(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.scatter(x, pe, s=30, alpha=0.9, color='green')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[8,10,26,33,54,61] #manually enter these values, this information is in the meta data by birth weight
    plt.plot(4, pe[4],'ro') #4 is the array index of the persistence diagram list
    plt.plot(6, pe[6],'ro') #'pe' stands for persistence entropy 
    plt.plot(16, pe[16],'ro') 
    plt.plot(18, pe[18],'ro')
    plt.plot(27, pe[27],'ro')
    plt.plot(31, pe[31],'ro')

    if(dim==0): 
        plt.plot(36, 8.7,'go') #green
        plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 8.7,"normal pregnacy")
        plt.text(37, 8.65,"bad pregnacy: birth weight")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 7.55,'go') #green
        plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 7.55,"normal pregnacy")
        plt.text(37, 7.50,"bad pregnacy: birth weight")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#based off disease
def save_persistence_entropy_disease(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.scatter(x, pe, s=30, alpha=0.9, color='green')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[6, 10, 12, 14, 15, 16, 33, 36, 41, 54, 60, 71, 75, 82, 87] #manually enter these values, this information is in the meta data by birth weight
    plt.plot(3, pe[3],'ro') #4 is the array index of the persistence diagram list
    plt.plot(6, pe[6],'ro') #'pe' stands for persistence entropy 
    plt.plot(7, pe[7],'ro')
    plt.plot(9, pe[9],'ro')
    plt.plot(10, pe[10],'ro')
    plt.plot(18, pe[18],'ro') #w16
    plt.plot(19, pe[19],'ro')
    plt.plot(22, pe[22],'ro') #w41
    plt.plot(27, pe[27],'ro')
    plt.plot(30, pe[30],'ro')
    plt.plot(31, pe[31],'ro')
    plt.plot(33, pe[33],'ro')
    plt.plot(34, pe[34],'ro')
    plt.plot(38, pe[38],'ro')
    plt.plot(42, pe[42],'ro') 

    if(dim==0): 
        plt.plot(36, 8.7,'go') #green
        plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 8.7,"normal pregnacy")
        plt.text(37, 8.65,"bad pregnacy: disease")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 7.55,'go') #green
        plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 7.55,"normal pregnacy")
        plt.text(37, 7.50,"bad pregnacy: disease")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#based off age of discharge
def save_persistence_entropy_age(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.scatter(x, pe, s=30, alpha=0.9, color='green')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[8, 12, 17, 26, 32, 41, 54, 61] #manually enter these values, this information is in the meta data by birth weight
    plt.plot(4, pe[4],'ro') #4 is the array index of the persistence diagram list
    plt.plot(7, pe[7],'ro')
    plt.plot(11, pe[11],'ro')
    plt.plot(16, pe[16],'ro')
    plt.plot(17, pe[17],'ro')
    plt.plot(22, pe[22],'ro')
    plt.plot(27, pe[27],'ro')
    plt.plot(31, pe[31],'ro')

    
    if(dim==0): 
        plt.plot(36, 8.7,'go') #green
        plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 8.7,"normal pregnacy")
        plt.text(37, 8.65,"bad pregnacy: age of discharge")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 7.55,'go') #green
        plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 7.55,"normal pregnacy")
        plt.text(37, 7.50,"bad pregnacy: age of discharge")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#based off birth length 
def save_persistence_entropy_birthlength(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.scatter(x, pe, s=30, alpha=0.9, color='green')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    bad_set=[8, 10, 12, 33, 41, 54,61] #manually enter these values, this information is in the meta data by birth weight
    plt.plot(4, pe[4],'ro') #4 is the array index of the persistence diagram list
    plt.plot(6, pe[6],'ro') #'pe' stands for persistence entropy 
    plt.plot(16, pe[16],'ro') 
    plt.plot(18, pe[18],'ro')
    plt.plot(22, pe[22],'ro')
    plt.plot(27, pe[27],'ro')
    plt.plot(31, pe[31],'ro')
    
    if(dim==0): 
        plt.plot(36, 8.7,'go') #green
        plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 8.7,"normal pregnacy")
        plt.text(37, 8.65,"bad pregnacy: birth length")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 7.55,'go') #green
        plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 7.55,"normal pregnacy")
        plt.text(37, 7.50,"bad pregnacy: birth length")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#based off gestational weeks
def save_persistence_entropy_gweeks(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.scatter(x, pe, s=30, alpha=0.9, color='green')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    for i, txt in enumerate(women_set):
        plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
    
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    #for G22
    #bad_set=[8, 10, 12, 14, 32, 33, 36, 38, 53, 54, 71] #manually enter these values, this information is in the meta data by birth weight
    #for G32
    plt.plot(4, pe[4],'ro') #4 is the array index of the persistence diagram list
    plt.plot(6, pe[6],'ro') #'pe' stands for persistence entropy 
    #plt.plot(17, pe[17],'ro') #w26 was dropped
    plt.plot(7, pe[7],'ro')
    plt.plot(9, pe[9],'ro')
    plt.plot(17, pe[17],'ro')
    plt.plot(18, pe[18],'ro')
    plt.plot(19, pe[19],'ro')
    plt.plot(20, pe[20],'ro')
    plt.plot(26, pe[26],'ro')
    plt.plot(27, pe[27],'ro')
    plt.plot(33, pe[33],'ro')
    
    if(dim==0): 
        plt.plot(36, 8.7,'go') #green
        plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        plt.text(37, 8.7,"normal pregnacy")
        plt.text(37, 8.65,"bad pregnacy: gestational weeks")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    elif(dim==1):
        plt.plot(36, 7.55,'go') #green
        plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        plt.text(37, 7.55,"normal pregnacy")
        plt.text(37, 7.50,"bad pregnacy: gestational weeks")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#based off all features
def save_persistence_entropy(pe, file_name, dim):
    print ("Saving Persistence Entropy in PNG ")
    x=linspace(0,num_of_women-1, num=num_of_women) # good for both dim0 and dim1

    plt.figure(figsize=(15, 8))
    plt.title("women persistence entropy dimension %i"%dim)
    plt.xlabel('persistence diagram index')
    plt.ylabel('Persistence Entropy')
    plt.ylim(7.8,8.5)
    plt.scatter(x, pe, s=200, alpha=1, color='limegreen')
     #plt.axhline(y=6.77, color='b', linestyle='-')
        
    #for i, txt in enumerate(women_set):
    #    plt.annotate(txt, (x[i], pe[i]))

    #normal_set = get_normal_set(Gwk_normal)   from Gwk_normal get the normal_set function
 
    #bad_set = get_bad_set(Gwk_bad)   from Gwk_bad get the bad_set function
    #for G22 #both have 21 bad women
    #bad_set=[6, 8, 10, 12, 14, 16, 17, 26, 32, 33, 36, 38, 41, 53, 54, 60, 61, 71, 75, 82, 87] #manually enter these values
    #for G32
    bad_set=[6, 8, 10, 12, 14, 15, 16, 17, 32, 33, 36, 38, 41, 53, 54, 60, 61, 71, 75, 82, 87] #manually enter these values
    #for G22

    plt.scatter(3, pe[3], s = 200, alpha = 1, color ='salmon') #6
    plt.scatter(4, pe[4], s = 200, alpha = 1, color ='salmon') #8  #'pe' stands for persistence entropy 
    plt.scatter(6, pe[6],s = 200, alpha = 1, color ='salmon') #10
    plt.scatter(7, pe[7],s = 200, alpha = 1, color ='salmon') #12
    plt.scatter(8, pe[8],s = 200, alpha = 1, color ='salmon') #14
    plt.scatter(9, pe[9],s = 200, alpha = 1, color ='salmon') #16
    plt.scatter(10, pe[10],s = 200, alpha = 1, color ='salmon') #17
    plt.scatter(15, pe[15],s = 200, alpha = 1, color ='salmon') #26
    plt.scatter(16, pe[16],s = 200, alpha = 1, color ='salmon') #32
    plt.scatter(17, pe[17],s = 200, alpha = 1, color ='salmon') #33
    plt.scatter(18, pe[18],s = 200, alpha = 1, color ='salmon') #36
    plt.scatter(19, pe[19],s = 200, alpha = 1, color ='salmon') #38
    plt.scatter(21, pe[21],s = 200, alpha = 1, color ='salmon') #41
    plt.scatter(25, pe[25],s = 200, alpha = 1, color ='salmon') #53
    plt.scatter(26, pe[26],s = 200, alpha = 1, color ='salmon') #54
    plt.scatter(28, pe[28],s = 200, alpha = 1, color ='salmon') #60
    plt.scatter(29, pe[29],s = 200, alpha = 1, color ='salmon') #61
    plt.scatter(31, pe[31],s = 200, alpha = 1, color ='salmon') #71
    plt.scatter(32, pe[32],s = 200, alpha = 1, color ='salmon') #75
    plt.scatter(36, pe[36],s = 200, alpha = 1, color ='salmon') #82
    plt.scatter(39, pe[39],s = 200, alpha = 1, color ='salmon') #87
        
    #for G32
    #plt.scatter(3, pe[3], s = 200, alpha = 1, color ='salmon') #6
    #plt.scatter(4, pe[4], s = 200, alpha = 1, color ='salmon') #8  #'pe' stands for persistence entropy 
    #plt.scatter(6, pe[6],s = 200, alpha = 1, color ='salmon') #10
    #plt.scatter(7, pe[7],s = 200, alpha = 1, color ='salmon') #12
    #plt.scatter(9, pe[9],s = 200, alpha = 1, color ='salmon') #14
    #plt.scatter(10, pe[10],s = 200, alpha = 1, color ='salmon') #15
    #plt.scatter(11, pe[11],s = 200, alpha = 1, color ='salmon') #16
    #plt.scatter(12, pe[12],s = 200, alpha = 1, color ='salmon') #17
    #plt.scatter(17, pe[17],s = 200, alpha = 1, color ='salmon') #32
    #plt.scatter(18, pe[18],s = 200, alpha = 1, color ='salmon') #33
    #plt.scatter(19, pe[19],s = 200, alpha = 1, color ='salmon') #36
    #plt.scatter(20, pe[20],s = 200, alpha = 1, color ='salmon') #38
    #plt.scatter(22, pe[22],s = 200, alpha = 1, color ='salmon') #41
    #plt.scatter(26, pe[26],s = 200, alpha = 1, color ='salmon') #53
    #plt.scatter(27, pe[27],s = 200, alpha = 1, color ='salmon') #54
    #plt.scatter(30, pe[30],s = 200, alpha = 1, color ='salmon') #60
    #plt.scatter(31, pe[31],s = 200, alpha = 1, color ='salmon') #61
    #plt.scatter(33, pe[33],s = 200, alpha = 1, color ='salmon') #71
    #plt.scatter(34, pe[34],s = 200, alpha = 1, color ='salmon') #75
    #plt.scatter(38, pe[38],s = 200, alpha = 1, color ='salmon') #82
    #plt.scatter(42, pe[42],s = 200, alpha = 1, color ='salmon') #87
    
    
    #Unedited...
    #plt.plot(3, pe[3],'ro') 
    #
    #plt.plot(6, pe[6],'ro') #'pe' stands for persistence entropy 
    #plt.plot(7, pe[7],'ro')
    #plt.plot(9, pe[9],'ro')
    #plt.plot(10, pe[10],'ro')
    #plt.plot(11, pe[11],'ro')
    #plt.plot(16, pe[16],'ro')
    #plt.plot(17, pe[17],'ro')
    # plt.plot(18, pe[18],'ro')
    # plt.plot(19, pe[19],'ro')
    #plt.plot(20, pe[20],'ro')
    #plt.plot(22, pe[22],'ro')
    #plt.plot(26, pe[26],'ro')
    #plt.plot(27, pe[27],'ro')
    #plt.plot(30, pe[30],'ro')
    #plt.plot(31, pe[31],'ro')
    #plt.plot(33, pe[33],'ro')
    #plt.plot(34, pe[34],'ro')
    #plt.plot(38, pe[38],'ro')
    #plt.plot(42, pe[42],'ro')
        

   # if(dim==0): 
        #plt.plot(36, 8.7,'go') #green
        #plt.plot(36, 8.65,'ro') #red
        #plt.plot(36, 8.60,'ko') #black

        #plt.text(37, 8.7,"normal pregnacy")
        #plt.text(37, 8.65,"bad pregnacy: all features")
        #plt.text(37, 8.60,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
        
    #elif(dim==1):
        #plt.plot(36, 7.55,'go') #green
        #plt.plot(36, 7.50,'ro') #red
        #plt.plot(36, 7.45,'ko') #black

        #plt.text(37, 7.55,"normal pregnacy")
        #plt.text(37, 7.50,"bad pregnacy: all features")
        #plt.text(37, 7.40,"missing birth data") #missing data was dropped

        #comment this to save into file and .....
        #plt.show()
    
    print(pe)
    plt.savefig(os.path.join(my_path,  file_name))
    plt.close()

In [ ]:
#STEP ONE
#select which week data 

#WEEK 22
#Whole week
#Gwk="gw22.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/"

#Day one
Gwk="gw22_day1.csv"
my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day1/"

#Day two 
#Gwk="gw22_day2.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day2/"

#Day three
#Gwk="gw22_day3.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day3/"

#Day four
#Gwk="gw22_day4.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day4/"

#Day five
#Gwk="gw22_day5.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day5/"
 

#Day six
#Gwk="gw22_day6.csv"
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_22/week22_day6/"


#WEEK 32
#Whole week
#Gwk="gw32.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/"

#Day one
#Gwk="gw32_day1.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day1/"

#Day two 
#Gwk="gw32_day2.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day2/"

#Day three
#Gwk="gw32_day3.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day3/"

#Day four
#Gwk="gw32_day4.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day4/"

#Day five 
#Gwk="gw32_day5.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day5/"

#Day six
#Gwk="gw32_day6.csv" 
#my_path ="/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/resample_dim1_extended/week_32/week32_day6/"

rows_fullData, hdrs = get_data(Gwk) #get the entire data set  6 days --> 144 hrs
rowLen_full = len(rows_fullData) #full size number of rows  
colLen = len(hdrs) #number of collumns

#col 0 ---> hour
#col 1 ---> time
#last col--> AVERAGE

num_of_women = colLen -4
women_set=hdrs[2:colLen-2]
#subsample_pts=8640 #subsampled from 17281 points to 8640 for Greedy computation
subsample_pts = 2000 #subsampled from 2881 points to 2000

print("my_path: ", my_path)        
print("loading file: ",Gwk)
print("Headers: ",hdrs)
print("number of collumns: ",colLen)
print("number of rows: ",len(rows_fullData))
print("number of women: ",num_of_women)
print("women set: ", women_set)
   

In [ ]:
#STEP TWO

#compute persistence diagrams for all women using greedy permutation, and get the resampled greedy permutation points
pd0_list,pd1_list, perm0_list, perm1_list = get_all_persistence_diagrams_using_greedy_permutation()
#pd0_list, perm0_list= get_all_persistence_diagrams_using_greedy_permutation()



In [ ]:
#step three 
#save persistence diagrams
save_all_persistence_diagrams(pd0_list,pd1_list)

#save scatter plots png files
save_all_scatter_plots()

#save permutations points onto a spread sheet

In [ ]:
#getting the scater plots of the data before and after the greedy permutation
#two seperate figures
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 10})
w = 'w5'

row_data=get_clean_data(w) 
X = get_X_data_points(row_data, w)

dgms_sub, idx_perm, r_cover = get_rips_pd_greedyPermutation(X)

plt.figure(figsize=(3, 3))
#plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], color = 'gray',  edgecolor = 'black')
#plt.title("Original Point Cloud (%i Points)"%(X.shape[0]))
plt.xlabel('Hours')
plt.ylabel('Activity')
plt.savefig('resampled_0.png', dpi=600, bbox_inches = "tight",facecolor='w')

#plt.subplot(222)
plt.figure(figsize=(3, 3))
plt.scatter(X[idx_perm,0], X[idx_perm,1],  color = 'dimgray', edgecolor = 'black')
#plt.title("Greedy Subsampled Cloud (%i Points)"%(idx_perm.size))
plt.xlabel('Hours')
plt.ylabel('Activity')
plt.savefig('resampled_1.png', dpi=600, bbox_inches = "tight",facecolor='w')

#take off titles 
#size 10 font
 #need for one day too

In [ ]:
def greedyPointsToXcel(fname):
    xcelFile = ''.join([my_path, fname]) #mypath is a global variable
   
    df = pd.DataFrame() #create empty data frame
 
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(xcelFile, engine='xlsxwriter')
   
    print ("Computing subsample points for: ", fname )
    for w in (women_set):
        t= w.replace('w','t')  #add time collumn string
        print ("Computing subsample points for w:  ", w )
        row_data=get_clean_data(w) #skip row with empty cells     
        X= get_X_data_points(row_data, w)
        [dgms_sub, idx_perm, r_cover] = get_rips_pd_greedyPermutation(X)
         
        df[t] = X[idx_perm,0]  #add time column
        df[w]=X[idx_perm,1]         #add value column
   
  
    df.to_excel(writer, sheet_name='Sheet1', index=False)    # Convert the dataframe to an XlsxWriter Excel object.
    writer.save() # Close the Pandas Excel writer and output the Excel file.
 
    return

In [ ]:
#call the function to get the excel files 

#Day one
xcelFile='greedyPoints_day1.xlsx'
greedyPointsToXcel(xcelFile)
 
#Day two
#xcelFile='greedyPoints_day2.xlsx'
#greedyPointsToXcel(xcelFile)
 
#Day three
#xcelFile='greedyPoints_day3.xlsx'
#greedyPointsToXcel(xcelFile)

#Day four
#xcelFile='greedyPoints_day4.xlsx'
#greedyPointsToXcel(xcelFile)

#Day five
#xcelFile='greedyPoints_day5.xlsx'
#greedyPointsToXcel(xcelFile)

#Day six
#xcelFile='greedyPoints_day6.xlsx'
#greedyPointsToXcel(xcelFile)
 

In [ ]:
#compute bottleneck distances for all women dim 0 and dim 1

BN_dist_all_women_D0 = compute_BN_distance_all_women(pd0_list)
BN_dist_all_women_D1 = compute_BN_distance_all_women(pd1_list)

In [ ]:
#save bottleneck distance matrix for all women
   
fname0='BottleNeck_Distance_D0.csv' 
fname1='BottleNeck_Distance_D1.csv'

rowids = women_set.copy()
col_names=women_set.copy()
col_names.insert(0," ")

save_Bottleneck_dist(fname0,BN_dist_all_women_D0, col_names, rowids)  #for dim 0
save_Bottleneck_dist(fname1,BN_dist_all_women_D1, col_names, rowids) #for dim1


In [ ]:
#compute persistent entropy for all persistence diagrams  
e0 = persistent_entropy(pd0_list)
e1 = persistent_entropy(pd1_list)
print(e0)
print(e1)
w = ['w3', 'w4', 'w5', 'w6', 'w8', 'w9', 'w10', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w21', 'w23', 'w24', 'w25', 'w32', 'w33', 'w36', 'w38', 'w39', 'w41', 'w42', 'w44', 'w46', 'w53', 'w54', 'w57', 'w58', 'w60', 'w61', 'w66', 'w71', 'w75', 'w76', 'w79', 'w81', 'w82', 'w83', 'w84', 'w85', 'w87', 'w93']
col0 = 'Woman'
col1 = 'Dimension 0 Entropy'
col2 = 'Dimension 1 Entropy'
data = pd.DataFrame({col0:w,col1:e0,col2:e1})
#data = pd.DataFrame({col0:w,col1:e0})

#data.to_excel('/Users/sashielvagus/Desktop/Topology Project/RipserFiltration/boxplots_data/entropy_perday_GREEDY/Entropy_data_day6_GREEDY32.xlsx', sheet_name='day6', index=False)
              
              
              
              

In [ ]:
fn0="PersistenceEntropy_PSQI_D0_32"
fn1="PersistenceEntropy_PSQI_D1"

fn2="PersistenceEntropyGestationalWeeks_PSQI_D0"
fn3="PersistenceEntropyGestationalWeeks_PSQI_D1"

fn4="PersistenceEntropyBirthLength_PSQI_D0"
fn5="PersistenceEntropyBirthLength_PSQI_D1"

fn6="PersistenceEntropyAge_PSQI_D0"
fn7="PersistenceEntropyAge_PSQI_D1"

fn8="PersistenceEntropyDisease_PSQI_D0"
fn9="PersistenceEntropyDisease_PSQI_D1"

fn10="PersistenceEntropyBirthWeight_PSQI_D0"
fn11="PersistenceEntropyBirthWeight_PSQI_D1"

#All features
save_persistence_entropy(e0, fn0, 0) #dimension 0
save_persistence_entropy(e1, fn1, 1) #dimension 1

#based off gestational weeks
save_persistence_entropy_gweeks(e0, fn2, 0)
save_persistence_entropy_gweeks(e1, fn3, 1)

#based off birth length 
save_persistence_entropy_birthlength(e0, fn4, 0)
save_persistence_entropy_birthlength(e1, fn5, 1)

#based off age of discharge
save_persistence_entropy_age(e0, fn6, 0)
save_persistence_entropy_age(e1, fn7, 1)

#based off disease
save_persistence_entropy_disease(e0, fn8, 0)
save_persistence_entropy_disease(e1, fn9, 1)

#based off birthweight
save_persistence_entropy_birthweight(e0, fn10, 0)
save_persistence_entropy_birthweight(e1, fn11, 1)

In [ ]:
#Plotting Entropy for week G22
#G22
#plot data
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
datas = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet2') 
entropy_inc_all = datas['Entropy']
b_inc_all = datas['Woman']

#plot only the bad data 
datas_2 = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet3')
entropy_bad = datas_2['Entropy']
b_bad = datas_2['Woman']

plt.figure(figsize=(7,3.25))
#fix woman index and ticks parameters 
plt.xlabel('Gestational week 22', size = 10)  #ylabel and size
plt.ylabel('Entropy value', size = 10)

plt.tick_params(axis='x', labelrotation=45, labelsize=8)
plt.tick_params(axis='y',  labelsize=8)

plt.scatter(b_inc_all, entropy_inc_all, s=20, alpha=1, c = '#1A85FF')
plt.scatter(b_bad, entropy_bad, s=20, alpha=1, c = '#D41159')
plt.legend(["Uncomplicated", "Complicated"],prop = { "size": 10 })
plt.axhline(y = 8.22, color = "grey", linestyle = '--', dashes=(5, 5)) #entropy level 8.25 #loosley dashed line
plt.savefig('bad_v_entropy_22.png',dpi=600, bbox_inches = "tight", facecolor='w')
plt.show()


In [ ]:
#Plotting Entropy for week G32
#G32
#plot data
plt.rcParams["font.family"] = "Times New Roman"
datas = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet4') 
entropy_inc_all = datas['Entropy']
b_inc_all = datas['Woman']

#plot only the bad data 
datas_2 = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet5')
entropy_bad = datas_2['Entropy']
b_bad = datas_2['Woman']

plt.figure(figsize=(7,3.25))
#fix woman index and ticks parameters 
plt.xlabel('Gestational week 32', size = 10)  #ylabel and size
plt.ylabel('Entropy value', size = 10)

plt.tick_params(axis='x', labelrotation=45, labelsize=8)
plt.tick_params(axis='y',  labelsize=8)

plt.scatter(b_inc_all, entropy_inc_all, s=20, alpha=1, c = '#1A85FF')
plt.scatter(b_bad, entropy_bad, s=20, alpha=1, c = '#D41159')
plt.legend(["Uncomplicated", "Complicated"],prop = { "size": 10 })
plt.axhline(y = 8.22, color = "grey", linestyle = '--', dashes=(5, 5)) #entropy level 8.25 #loosley dashed line
plt.savefig('bad_v_entropy_32.png',dpi=600, bbox_inches = "tight", facecolor='w')
plt.show()

In [ ]:
#Plotting BMI versus entropy 
#G22
#plot data
plt.rcParams["font.family"] = "Times New Roman"
datas = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet2') 
entropy_inc_all = datas['Entropy']
bmi_inc_all = datas['BMI']
#plot only the bad data 
datas_2 = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet3')
entropy_bad = datas_2['Entropy']
bmi_bad = datas_2['BMI']

plt.figure(figsize=(7,3.25))

plt.xlabel('BMI', size = 10)  #ylabel and size
plt.ylabel('Entropy', size = 10)
plt.scatter(bmi_inc_all, entropy_inc_all, s=20, alpha=1, c = '#1A85FF')
plt.scatter(bmi_bad, entropy_bad, s=20, alpha=1, c = '#D41159')
plt.legend(["Uncomplicated", "Complicated"],prop = { "size": 10 })
plt.axhline(y = 8.25, color = "grey", linestyle = '--', dashes=(5, 5)) #entropy level 8.25 #loosley dashed line
plt.axvline(x = 24.9, color = "grey", linestyle = '--', dashes=(5, 5)) #normal bmi 18.5-24.9 according to CDC
plt.savefig('bmi_v_entropy.png',dpi=600, bbox_inches = "tight", facecolor='w')
plt.show()



In [ ]:
#Plotting women versus glucose
#G22
plt.rcParams["font.family"] = "Times New Roman"
datass = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet2') 
entropy_g = datass['Entropy_g']
glucose_g = datass['Blood Glucose']

datas_2 = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet3')
entropy_b = datas_2['Entropy_b']
glucose_b = datas_2['Blood Glucose']

plt.figure(figsize=(7,3.25))
plt.xlabel('Blood glucose (mg/dL)', size = 10)  #ylabel and size
plt.ylabel('Entropy', size = 10)
plt.scatter(glucose_g, entropy_g, s=20, alpha=1, c = "#1A85FF")
plt.scatter(glucose_b, entropy_b, s=20, alpha=1, c = "#D41159")
plt.legend(["Uncomplicated", "Complicated"],prop = { "size": 10 })
plt.axhline(y = 8.25, color = "grey", linestyle = '--', dashes=(5, 5))
plt.axvline(x = 95, color = "grey", linestyle = '--', dashes=(5, 5)) #target for women testing blood glucose levels during pregnancy:  95 mg/dl 
# according to american diabetes association
plt.savefig('glucose_v_entropy.png',dpi=600, bbox_inches = "tight",facecolor='w')
plt.show()



In [ ]:
#Plotting women versus gestational weeks
#G22
plt.rcParams["font.family"] = "Times New Roman"
datasss = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet2') 
entropie = datasss['Entropy']
gest_g = datasss['Gestational age of birth']

datas_3 = pd.read_excel('/Users/sashielvagus/Desktop/GOOD_V_BAD.xlsx','Sheet3')
entropieb = datas_3['Entropy']
gest_b = datas_3['Gestational age of birth']

plt.figure(figsize=(7,3.25))
plt.xlabel('Gestational age of birth', size = 10)  #ylabel and size
plt.ylabel('Entropy', size = 10)
plt.scatter(gest_g, entropie, s=20, alpha=1, c = "#1A85FF")
plt.scatter(gest_b, entropieb, s=20, alpha=1, c = "#D41159")
plt.legend(["Uncomplicated", "Complicated"],prop = { "size": 10 })
plt.axhline(y = 8.25, color = "grey", linestyle = '--', dashes=(5, 5))
#plt.axvline(x = 40, color = "black", linestyle = '-') #normal pregnancy is 40 weeks 
#plt.axvline(x = 38, color = "black", linestyle = '-')
#plt.axvline(x = 42, color = "black", linestyle = '-')
plt.savefig('gestationalage_v_entropy.png',dpi=600, bbox_inches = "tight",facecolor='w')
plt.show()



In [ ]:
#get persistence diagram for AVERAGE
w='AVERAGE'
row_data=get_clean_data(w)   
pts= get_X_data_points(row_data, w)
[dgms_sub, idx_perm, r_cover] = get_rips_pd_greedyPermutation(pts)
pd0_avg=dgms_sub[0]
pd1_avg=dgms_sub[1]
plt.figure(figsize=(15,8))
plt.title("Persistence Diagram For Average" ) 
persim.plot_diagrams([pd0_avg,pd1_avg],labels=['$H_0$', '$H_1$'])

In [ ]:
#get persistence diagram for MEDIAN
w='MEDIAN'
row_data=get_clean_data(w)   
pts= get_X_data_points(row_data, w)
[dgms_sub, idx_perm, r_cover] = get_rips_pd_greedyPermutation(pts)
pd0_median=dgms_sub[0]
pd1_median=dgms_sub[1]
plt.figure(figsize=(15,8))
plt.title("Persistence Diagram For Median" ) 
persim.plot_diagrams([pd0_median,pd1_median],labels=['$H_0$', '$H_1$'])

In [ ]:
#Average

bnd_avg0 = calculate_BN_dist_women_against_avg(pd0_avg, pd0_list)
bnd_avg1 = calculate_BN_dist_women_against_avg(pd1_avg, pd1_list)

In [ ]:
#Median

bnd_median0 = calculate_BN_dist_women_against_median(pd0_median, pd0_list)
bnd_median1 = calculate_BN_dist_women_against_median(pd1_median, pd1_list)

In [ ]:
fn0="BottleNeckAgainstAverage_PSQI_D0"
fn1="BottleNeckAgainstAverage_PSQI_D1"


plot_BN_women_against_avg(bnd_avg0, fn0 ,0)
plot_BN_women_against_avg(bnd_avg1, fn1 ,1)

In [ ]:
fn2="BottleNeckAgainstMedian_PSQI_D0"
fn3="BottleNeckAgainstMedian_PSQI_D1"


plot_BN_women_against_median(bnd_median0, fn2 ,0)
plot_BN_women_against_median(bnd_median1, fn3 ,1)